In [6]:
#importy
import pandas as pd
import requests
from bs4 import BeautifulSoup as bs
import re

In [7]:
#tworze pusty dataframe
df = pd.DataFrame(columns=['tytul','typ','miejsce','pokoje','powierzchnia','cena'])
df

,tytul,typ,miejsce,pokoje,powierzchnia,cena


In [132]:
def poczatek():
    url1 = input('Wprowadź link do strony z wynikami Twojego wyszukiwania')
    strona = requests.get(url)
    soup = bs(strona.text,'lxml')
    
    return soup, url1

#soup, url = poczatek()

In [133]:
soup, url1 = poczatek()

Wprowadź link do strony z wynikami Twojego wyszukiwaniahttps://www.otodom.pl/wynajem/mieszkanie/tychy/?search%5Bfilter_float_m%3Ato%5D=50&search%5Bcity_id%5D=166


In [135]:
url1

'https://www.otodom.pl/wynajem/mieszkanie/tychy/?search%5Bfilter_float_m%3Ato%5D=50&search%5Bcity_id%5D=166'

In [136]:
soup

<!DOCTYPE html>
<html xmlns:fb="http://www.facebook.com/2008/fbml" xmlns:og="http://ogp.me/ns#">
<head>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<title>Mieszkanie  na wynajem w Tychy, śląskie - www.otodom.pl</title>
<link href="https://www.otodom.pl/wynajem/mieszkanie/tychy/?search%5Bfilter_float_m%3Ato%5D=50&amp;search%5Bcity_id%5D=166" rel="prev"/>
<meta content="index, follow" name="robots"/> <link href="https://www.otodom.pl/wynajem/mieszkanie/tychy/?page=2" rel="canonical"/> <meta content="pl" http-equiv="Content-Language"/>
<meta content="Zobacz 43 mieszkań do wynajęcia z co najmniej 1 pokojem w miejscowości Tychy, śląskie od 500. Znajdź te i wiele innych ofert z rynku nieruchomości w kategorii mieszkania na sprzedaż, ale także sprzedaży i wynajmu domów, działek, pokojów, lokali usługowych, biur, garaży i magazynów w otodom.pl. Najpopularniejszym serwisie nieruchomości w Polsce." name="description"/>
<meta content="width=device-width, initial-scale=1.0

In [59]:
def zakres():    
    p2 = soup.find('form', {"id":'pagerForm'})
    p2 = p2.find_all('a',"")

    strony = []

    for a in p2:
        strona = a.text
        if strona != "":
            strona = int(strona)
            strony.append(strona)
        else:
            pass
    strona_max = max(strony)
    strona_min = min(strony)

    return(strona_min, strona_max)

#strona_min, strona_max = zakres()

In [137]:
strona_min,strona_max = zakres()

In [138]:
strona_min

1

In [139]:
strona_max

2

In [63]:
def limit_wyszukiwan(strona_max):

    limit_wyszukiwania = 0
    
    if strona_max > 50:
        limit_wyszukiwania =+ 50
    else:
        limit_wyszukiwania =+ strona_max
    
    return limit_wyszukiwania

In [140]:
limit_wyszukiwania = limit_wyszukiwan(strona_max)
limit_wyszukiwania

2

In [146]:
def parametry():
    
    df = pd.DataFrame(columns=['tytul','typ','miejsce','pokoje','powierzchnia','cena'])
    
    for i in range(strona_min,limit_wyszukiwania+1):
        
    
        l_tytul = []
        l_typ = []
        l_miejsce = []
        l_pokoje = []
        l_powierzchnia = []
        l_cena = []
        l_wynajmujacy = []
        ##lista_list = [l_tytul,l_typ,l_miejsce,l_pokoje,l_powierzchnia,l_cena,l_wynajmujacy]
        
        url = url1+ f'&page={i}'
        strona = requests.get(url)
        soup = bs(strona.text,'lxml')

        oferty = soup.find_all('article')
    

        for oferta in oferty:


            tytul = oferta.find('span', {"class":"offer-item-title"}).text
            l_tytul.append(tytul)

            typ = oferta.find('span', {"class": "hidden-xs"}).text.strip().rstrip(":")
            l_typ.append(typ)

            miejsce = oferta.find('p', {"class": "text-nowrap"}).text
            miejsce = miejsce[miejsce.find(":",0,-1)+2:]
            l_miejsce.append(miejsce)

            pokoje = int(oferta.find('li', {'class': 'offer-item-rooms hidden-xs'}).text[0])
            l_pokoje.append(pokoje)

            powierzchnia = int(re.findall(r'\d+',oferta.find('li', {"class": "hidden-xs offer-item-area"}).text)[0])
            l_powierzchnia.append(powierzchnia)

            cena = int(re.findall(r'\d+',re.sub(r'\s+','',oferta.find('li', {'class': "offer-item-price"}).text.strip()))[0])
            l_cena.append(cena)

            wynajmujacy = oferta.find('ul', {'class': "params-small clearfix hidden-xs"}).text.strip()
            l_wynajmujacy.append(wynajmujacy)

        '''df = pd.DataFrame(columns=['tytul','typ','miejsce','pokoje','powierzchnia','cena','wynajmujacy'])
        df.append(l_tytul)'''
    
        df1 = pd.DataFrame({'tytul':l_tytul, 
                                 "typ":l_typ,
                                 "miejsce": l_miejsce,
                                 "pokoje": l_pokoje,
                                 "powierzchnia": l_powierzchnia,
                                 "cena": l_cena,
                                 "wynajmujacy": l_wynajmujacy}) 
        ramki = [df,df1]
        df = pd.concat(ramki)
    return df
    
    #return df

In [147]:
parametry()

,tytul,typ,miejsce,pokoje,powierzchnia,cena,wynajmujacy
0,"TYCHY - najpiękniejsze, reprezentacyjne mieszk...",Mieszkanie na wynajem,"Tychy, śląskie",2,42,2200,Oferta prywatna
1,Nowoczesne mieszkanie - Strefa Ekonomiczna Tychy,Mieszkanie na wynajem,"Tychy, śląskie",2,39,1600,Oferta prywatna
2,Wynajmę bezpośrednio,Mieszkanie na wynajem,"Tychy, śląskie",2,36,1500,Obsługa zdalna\n\n ...
3,Mieszkanie do wynajęcia ul Skalna,Mieszkanie na wynajem,"Tychy, śląskie",2,40,2000,MS HOME Skaba
4,Mieszkanie w nowym bloku z windą - czynsz w cenie,Mieszkanie na wynajem,"Tychy, Żwaków",2,41,1800,Aston Biuro Nieruchomości
5,"TYCHY - najpiękniejsze, reprezentacyjne mieszk...",Mieszkanie na wynajem,"Tychy, śląskie",2,42,2200,Oferta prywatna
6,"Wynajem mieszkania 48mkw, ul. Konecznego",Mieszkanie na wynajem,"Tychy, śląskie",2,48,1709,Asset Nieruchomości- POSZUKUJE MIESZKAŃ i DOMÓW.
7,kawalerka w Tychach,Mieszkanie na wynajem,"Tychy, śląskie",1,38,1400,Oferta prywatna
8,"Mieszkanie 42m2, 2 pokoje, dla pierwszego loka...",Mieszkanie na wynajem,"Tychy, śląskie",2,41,1700,Oferta prywatna
9,Nowoczesne mieszkanie - Strefa Ekonomiczna Tychy,Mieszkanie na wynajem,"Tychy, śląskie",2,39,1600,Oferta prywatna


In [142]:
tabela = parametry()

In [143]:
tabela

,tytul,typ,miejsce,pokoje,powierzchnia,cena


In [120]:
for i in range(strona_min,limit_wyszukiwania+1):
    url = url1+ f'&page={i}'
    strona = requests.get(url)
    soup = bs(html,'lxml')
    
    print(url)
    print(strona)
    print(soup)

https://www.otodom.pl/wynajem/mieszkanie/tychy/?search%5Bfilter_float_m%3Ato%5D=50&search%5Bcity_id%5D=166&page=1
<Response [200]>
<!DOCTYPE html>
<html xmlns:fb="http://www.facebook.com/2008/fbml" xmlns:og="http://ogp.me/ns#">
<head>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<title>Mieszkanie  na wynajem w Tychy, śląskie - www.otodom.pl</title>
<link href="https://www.otodom.pl/wynajem/mieszkanie/tychy/?search%5Bfilter_float_m%3Ato%5D=50&amp;search%5Bcity_id%5D=166&amp;page=2" rel="next"/>
<meta content="index, follow" name="robots"/> <link href="https://www.otodom.pl/wynajem/mieszkanie/tychy/" rel="canonical"/> <meta content="pl" http-equiv="Content-Language"/>
<meta content="Zobacz 43 mieszkań do wynajęcia z co najmniej 1 pokojem w miejscowości Tychy, śląskie od 500. Znajdź te i wiele innych ofert z rynku nieruchomości w kategorii mieszkania na sprzedaż, ale także sprzedaży i wynajmu domów, działek, pokojów, lokali usługowych, biur, garaży i magazynów w ot

https://www.otodom.pl/wynajem/mieszkanie/tychy/?search%5Bfilter_float_m%3Ato%5D=50&search%5Bcity_id%5D=166&page=2
<Response [200]>
<!DOCTYPE html>
<html xmlns:fb="http://www.facebook.com/2008/fbml" xmlns:og="http://ogp.me/ns#">
<head>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<title>Mieszkanie  na wynajem w Tychy, śląskie - www.otodom.pl</title>
<link href="https://www.otodom.pl/wynajem/mieszkanie/tychy/?search%5Bfilter_float_m%3Ato%5D=50&amp;search%5Bcity_id%5D=166&amp;page=2" rel="next"/>
<meta content="index, follow" name="robots"/> <link href="https://www.otodom.pl/wynajem/mieszkanie/tychy/" rel="canonical"/> <meta content="pl" http-equiv="Content-Language"/>
<meta content="Zobacz 43 mieszkań do wynajęcia z co najmniej 1 pokojem w miejscowości Tychy, śląskie od 500. Znajdź te i wiele innych ofert z rynku nieruchomości w kategorii mieszkania na sprzedaż, ale także sprzedaży i wynajmu domów, działek, pokojów, lokali usługowych, biur, garaży i magazynów w ot

In [71]:
url = url1+ f'&page={i}'
url

'https://www.otodom.pl/wynajem/mieszkanie/tychy/?search%5Bfilter_float_m%3Ato%5D=50&search%5Bcity_id%5D=166&page=2'

In [8]:
#link
url = 'https://www.otodom.pl/wynajem/mieszkanie/tychy/?search%5Bfilter_float_m%3Ato%5D=50&search%5Bcity_id%5D=166'
strona = requests.get(url)

In [9]:
html = strona.content

In [10]:
soup = bs(html,'lxml')

In [93]:
with open("otodom.html",'wb') as plik:
    plik.write(soup.prettify(encoding='utf-8'))

In [95]:
for i in range(1,2+1):
    print(i)

1
2


### Tworzenie listy dostępnych stron z wynikami wyszukiwania.
I obsługi limitu wyszukiwań

In [8]:
'''
pagers = soup.find('ul',"pager")
pagers'''
#nieuzywane

'\npagers = soup.find(\'ul\',"pager")\npagers'

In [9]:
'''#znajdowanie aktualnego numeru strony
p = soup.find_all('a','active')
p
p[1].text
page_active = int(p[1].text)
page_active'''
#będzie niepotrzebne, kod poniżej tworzy listę stron.

"#znajdowanie aktualnego numeru strony\np = soup.find_all('a','active')\np\np[1].text\npage_active = int(p[1].text)\npage_active"

In [25]:
def zakres():    
    p2 = soup.find('form', {"id":'pagerForm'})
    p2 = p2.find_all('a',"")

    strony = []

    for a in p2:
        strona = a.text
        if strona != "":
            strona = int(strona)
            strony.append(strona)
        else:
            pass
    strona_max = max(strony)
    strona_min = min(strony)

    return(strona_min, strona_max)

In [29]:
strona_min, strona_max = zakres()

In [30]:
strona_min

1

In [31]:
strona_max

2

In [14]:
strona_max

2

In [17]:
for i in range(strona_min,strona_max+1):
    print(i)

1
2


In [ ]:
#główna pętla -> przechodzenie przez każdą stronę.


In [64]:
def limit_wyszukiwan(strona_max):

    limit_wyszukiwania = 0
    
    if strona_max > 50:
        limit_wyszukiwania =+ 50
    else:
        limit_wyszukiwania =+ strona_max
    
    return limit_wyszukiwania

In [65]:
limit_wyszukiwan(strona_max)

2


### Tworzenie pętli do spisania wszystkich ogłoszeń z strony

In [15]:
oferty = soup.find_all('article')

In [17]:
len(oferty)

27

In [18]:
oferty[0]

<article class="offer-item ad_id4a4Lw" data-featured-name="promo_top_ads" data-item-id="4a4Lw" data-tracking-id="61506945" data-url="https://www.otodom.pl/pl/oferta/tychy-najpiekniejsze-reprezentacyjne-mieszkanie-ID4a4Lw.html#71b39355d2" id="offer-item-ad_id4a4Lw">
<figure class="offer-item-image" data-featured-name="no_offer" data-quick-gallery='[{"photo":"https:\/\/ireland.apollo.olxcdn.com\/v1\/files\/eyJmbiI6ImV6YnEwZWF4cnhycjMtQVBMIiwidyI6W3siZm4iOiJqMWozbzEzbTZiZ24xLUFQTCIsInMiOiIxNCIsInAiOiIxMCwtMTAiLCJhIjoiMCJ9XX0.IfkLn0Sx_R55SgHJAo1Ltg3CCEwmflYm_oNq4pJwl8Q\/image;s=655x491;q=80","thumb":"https:\/\/ireland.apollo.olxcdn.com\/v1\/files\/eyJmbiI6ImV6YnEwZWF4cnhycjMtQVBMIiwidyI6W3siZm4iOiJqMWozbzEzbTZiZ24xLUFQTCIsInMiOiIxNCIsInAiOiIxMCwtMTAiLCJhIjoiMCJ9XX0.IfkLn0Sx_R55SgHJAo1Ltg3CCEwmflYm_oNq4pJwl8Q\/image;s=184x138;q=80"},{"photo":"https:\/\/ireland.apollo.olxcdn.com\/v1\/files\/eyJmbiI6InpucXJtdjdnanBrbDEtQVBMIiwidyI6W3siZm4iOiJqMWozbzEzbTZiZ24xLUFQTCIsInMiOiIxNCIsInAiOiIxMCwtMT

In [19]:
tytul = oferty[0].find('span', {"class":"offer-item-title"}).text
tytul

'TYCHY - najpiękniejsze, reprezentacyjne mieszkanie'

In [20]:
typ = oferty[0].find('span', {"class": "hidden-xs"}).text.strip().rstrip(":")
typ

'Mieszkanie na wynajem'

In [21]:
miejsce = oferty[0].find('p', {"class": "text-nowrap"}).text
miejsce = miejsce[miejsce.find(":",0,-1)+2:]
miejsce

'Tychy, śląskie'

In [22]:
pokoje = int(oferty[0].find('li', {'class': 'offer-item-rooms hidden-xs'}).text[0])
pokoje

2

In [23]:
powierzchnia = int(re.findall(r'\d+',oferty[0].find('li', {"class": "hidden-xs offer-item-area"}).text)[0])
powierzchnia

42

In [24]:
#cena = oferty[0].find('li', {'class': "offer-item-price"}).text.strip()
cena = int(re.findall(r'\d+',re.sub(r'\s+','',oferty[0].find('li', {'class': "offer-item-price"}).text.strip()))[0])
cena

2200

In [30]:
#wynajmujący
wynajmujacy = oferty[0].find('ul', {'class': "params-small clearfix hidden-xs"}).text.strip()
wynajmujacy

'Oferta prywatna'

In [90]:
#definiowanie pętli do sprawdzania parametrów
def parametry():
    
    
    l_tytul = []
    l_typ = []
    l_miejsce = []
    l_pokoje = []
    l_powierzchnia = []
    l_cena = []
    l_wynajmujacy = []
    ##lista_list = [l_tytul,l_typ,l_miejsce,l_pokoje,l_powierzchnia,l_cena,l_wynajmujacy]
    
    oferty = soup.find_all('article')
    

    for oferta in oferty:
        
        
        tytul = oferta.find('span', {"class":"offer-item-title"}).text
        l_tytul.append(tytul)
        
        typ = oferta.find('span', {"class": "hidden-xs"}).text.strip().rstrip(":")
        l_typ.append(typ)
        
        miejsce = oferta.find('p', {"class": "text-nowrap"}).text
        miejsce = miejsce[miejsce.find(":",0,-1)+2:]
        l_miejsce.append(miejsce)
        
        pokoje = int(oferta.find('li', {'class': 'offer-item-rooms hidden-xs'}).text[0])
        l_pokoje.append(pokoje)
        
        powierzchnia = int(re.findall(r'\d+',oferta.find('li', {"class": "hidden-xs offer-item-area"}).text)[0])
        l_powierzchnia.append(powierzchnia)
        
        cena = int(re.findall(r'\d+',re.sub(r'\s+','',oferta.find('li', {'class': "offer-item-price"}).text.strip()))[0])
        l_cena.append(cena)
        
        wynajmujacy = oferta.find('ul', {'class': "params-small clearfix hidden-xs"}).text.strip()
        l_wynajmujacy.append(wynajmujacy)
    
    '''df = pd.DataFrame(columns=['tytul','typ','miejsce','pokoje','powierzchnia','cena','wynajmujacy'])
    df.append(l_tytul)'''
    
        df1 = df = pd.DataFrame({'tytul':l_tytul, 
                                 "typ":l_typ,
                                 "miejsce": l_miejsce,
                                 "pokoje": l_pokoje,
                                 "powierzchnia": l_powierzchnia,
                                 "cena": l_cena,
                                 "wynajmujacy": l_wynajmujacy}) 
    
        ramki = []
    
    
     
    

In [94]:


tabela = parametry()
tabela

,tytul,typ,miejsce,pokoje,powierzchnia,cena,wynajmujacy
0,"TYCHY - najpiękniejsze, reprezentacyjne mieszk...",Mieszkanie na wynajem,"Tychy, śląskie",2,42,2200,Oferta prywatna
1,Nowoczesne mieszkanie - Strefa Ekonomiczna Tychy,Mieszkanie na wynajem,"Tychy, śląskie",2,39,1600,Oferta prywatna
2,Wynajmę bezpośrednio,Mieszkanie na wynajem,"Tychy, śląskie",2,36,1500,Obsługa zdalna\n\n ...
3,Mieszkanie do wynajęcia ul Skalna,Mieszkanie na wynajem,"Tychy, śląskie",2,40,2000,MS HOME Skaba
4,Mieszkanie w nowym bloku z windą - czynsz w cenie,Mieszkanie na wynajem,"Tychy, Żwaków",2,41,1800,Aston Biuro Nieruchomości
5,"TYCHY - najpiękniejsze, reprezentacyjne mieszk...",Mieszkanie na wynajem,"Tychy, śląskie",2,42,2200,Oferta prywatna
6,"Wynajem mieszkania 48mkw, ul. Konecznego",Mieszkanie na wynajem,"Tychy, śląskie",2,48,1709,Asset Nieruchomości- POSZUKUJE MIESZKAŃ i DOMÓW.
7,kawalerka w Tychach,Mieszkanie na wynajem,"Tychy, śląskie",1,38,1400,Oferta prywatna
8,"Mieszkanie 42m2, 2 pokoje, dla pierwszego loka...",Mieszkanie na wynajem,"Tychy, śląskie",2,41,1700,Oferta prywatna
9,Nowoczesne mieszkanie - Strefa Ekonomiczna Tychy,Mieszkanie na wynajem,"Tychy, śląskie",2,39,1600,Oferta prywatna


In [100]:
frames = [df,tabela]


,index,tytul,typ,miejsce,pokoje,powierzchnia,cena,wynajmujacy
0,0,"TYCHY - najpiękniejsze, reprezentacyjne mieszk...",Mieszkanie na wynajem,"Tychy, śląskie",2,42,2200,Oferta prywatna
1,1,Nowoczesne mieszkanie - Strefa Ekonomiczna Tychy,Mieszkanie na wynajem,"Tychy, śląskie",2,39,1600,Oferta prywatna
2,2,Wynajmę bezpośrednio,Mieszkanie na wynajem,"Tychy, śląskie",2,36,1500,Obsługa zdalna\n\n ...
3,3,Mieszkanie do wynajęcia ul Skalna,Mieszkanie na wynajem,"Tychy, śląskie",2,40,2000,MS HOME Skaba
4,4,Mieszkanie w nowym bloku z windą - czynsz w cenie,Mieszkanie na wynajem,"Tychy, Żwaków",2,41,1800,Aston Biuro Nieruchomości
...,...,...,...,...,...,...,...,...
76,22,Mieszkanie z ogródkiem do wynajęcia,Mieszkanie na wynajem,"Tychy, śląskie",2,46,2000,MS HOME Skaba
77,23,do wynajęcia mieszkanie 2 pokoje 4 Pory Roku,Mieszkanie na wynajem,"Tychy, śląskie",2,39,1550,Oferta pośrednika
78,24,Mieszkanie do wynajęcia dla pracowników firm !,Mieszkanie na wynajem,"Tychy, śląskie",2,39,3000,Nieruchomości Nobiles
79,25,"Tychy, umeblowane, 2 pok. 1 p, dobra lokalizacja",Mieszkanie na wynajem,"Tychy, C",2,38,1550,Oferta prywatna
